In [2]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv
import pandas as pd 
application_df = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_19/charity_data.csv") # Resources/charity_data.csv
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# preprocess the data as in deliverable 1
application_df.drop(columns=['EIN','NAME'], inplace=True)
appType_counts = application_df['APPLICATION_TYPE'].value_counts()
classification_counts = application_df['CLASSIFICATION'].value_counts()
replace_application = list(appType_counts[appType_counts < 500].index)
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
replace_class = list(classification_counts[classification_counts < 1000].index)
# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

application_cat= application_df.columns.drop(['ASK_AMT','IS_SUCCESSFUL'])
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

app_df = application_df[['STATUS','ASK_AMT','IS_SUCCESSFUL']]
app_df = pd.concat([app_df,encode_df],axis=1)
app_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
y = app_df['IS_SUCCESSFUL'].values
X = app_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [5]:
# Define the basic neural network model, use only two hidden layers
number_input_features = len(X_train[0])
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=number_input_features))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6489 - accuracy: 0.6730
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5795 - accuracy: 0.7208
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5675 - accuracy: 0.7234
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5634 - accuracy: 0.7253
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5612 - accuracy: 0.7252
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5601 - accuracy: 0.7270
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5590 - accuracy: 0.7273
Epoch 8/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5582 - accuracy: 0.7274
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5578 - accuracy: 0.7273
Epoch 10/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5575 - accuracy: 0.7281

In [6]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create callback that saves the models weight every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,save_freq='epoch',period=5)

In [7]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7312
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5524 - accuracy: 0.7320
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5523 - accuracy: 0.7324
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7318
Epoch 5/50
799/804 [============================>.] - ETA: 0s - loss: 0.5522 - accuracy: 0.7325
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7322
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7324
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7318
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7319
Epoch 9/50
804/804 [==============================] - 2s 2ms/s

In [8]:
# Only got ~72% accuracy for the above and need at least 75% accuracy, so optimize again with new model below
# try using three hidden layers instead of two and 100 epochs instead of 50, sigmoid instead of relu for other hidden layers
# keeping the same number of columns, bins etc to try getting more accuracy with all data
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=13, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=8,activation="sigmoid"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=5,activation="sigmoid"))
# output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 100 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.6162 - accuracy: 0.6938
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5716 - accuracy: 0.7302
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5649 - accuracy: 0.7318
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5598 - accuracy: 0.7325
Epoch 5/100
782/804 [============================>.] - ETA: 0s - loss: 0.5562 - accuracy: 0.7327
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7323
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5550 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5537 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5527 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] -

In [9]:
# Evaluate new model with scaled data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5532 - accuracy: 0.7268 - 450ms/epoch - 2ms/step
Loss: 0.5531617403030396, Accuracy: 0.7267638444900513


In [10]:
# got ~73% accuracy with new model, so trying again still with frequency of 10 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,save_freq='epoch',period=10)
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=13, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=8,activation="sigmoid"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=5,activation="sigmoid"))
# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 30 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=30,callbacks=[cp_callback])

Epoch 1/30
804/804 [==============================] - 2s 2ms/step - loss: 0.6390 - accuracy: 0.6475
Epoch 2/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5746 - accuracy: 0.7310
Epoch 3/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5708 - accuracy: 0.7320
Epoch 4/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5677 - accuracy: 0.7327
Epoch 5/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5641 - accuracy: 0.7327
Epoch 6/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5609 - accuracy: 0.7331
Epoch 7/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5572 - accuracy: 0.7326
Epoch 8/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5551 - accuracy: 0.7325
Epoch 9/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5539 - accuracy: 0.7320
Epoch 10/30
798/804 [============================>.] - ETA: 0s - loss: 0.5532 - accuracy: 0.7335
Epo

In [11]:
# Evaluate new model with scaled data, gives us only ~72% again
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5560 - accuracy: 0.7236 - 563ms/epoch - 2ms/step
Loss: 0.556021511554718, Accuracy: 0.7236151695251465


In [12]:
# Try changing activation to sigmoid for last hidden layer, frequency back to every 5 epochs
# changing hidden layers to relu and sigmoid with different number of neurons
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=2, save_weights_only=True,save_freq='epoch',period=5)
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=8, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=5,activation="relu"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=3,activation="sigmoid"))
# output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 50 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6061 - accuracy: 0.7030
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5745 - accuracy: 0.7277
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5709 - accuracy: 0.7288
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5688 - accuracy: 0.7293
Epoch 5/50
803/804 [============================>.] - ETA: 0s - loss: 0.5647 - accuracy: 0.7317
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5646 - accuracy: 0.7317
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5615 - accuracy: 0.7326
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5590 - accuracy: 0.7330
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7331
Epoch 9/50
804/804 [==============================] - 2s 2ms/s

In [13]:
# Evaluate accuracy and loss for this model, we get ~73% again
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5535 - accuracy: 0.7272 - 478ms/epoch - 2ms/step
Loss: 0.5535485148429871, Accuracy: 0.7272303104400635


In [14]:
# Try changing activation to all relu for the hidden layers, last attempt to improve accuracy
nn_new = tf.keras.models.Sequential()
# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=8, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=5,activation="relu"))
# Third hidden layer
nn_new.add(tf.keras.layers.Dense(units=3,activation="relu"))
# output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 100 epochs
fit_model = nn_new.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6257 - accuracy: 0.6950
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5824 - accuracy: 0.7285
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5710 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5659 - accuracy: 0.7302
Epoch 5/100
803/804 [============================>.] - ETA: 0s - loss: 0.5626 - accuracy: 0.7305
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5626 - accuracy: 0.7304
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5598 - accuracy: 0.7304
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7310
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5567 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] -

In [15]:
# Evaluate accuracy and loss for this model
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5540 - accuracy: 0.7258 - 503ms/epoch - 2ms/step
Loss: 0.5540044903755188, Accuracy: 0.7258309125900269


In [16]:
# Last model to optimize the data for maximum accuracy (best accuracy is between 72-73%)
number_input_features = len(X_train_scaled[0])
nn_last = tf.keras.models.Sequential()
# First hidden layer
nn_last.add(tf.keras.layers.Dense(units=8, input_dim = number_input_features,activation="relu"))
# Second hidden layer
nn_last.add(tf.keras.layers.Dense(units=5,activation="relu"))
# Third hidden layer
nn_last.add(tf.keras.layers.Dense(units=3,activation="sigmoid"))
# output layer
nn_last.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_last.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train new model with 50 epochs
fit_model = nn_last.fit(X_train_scaled, y_train, epochs=50,callbacks=[cp_callback])
model_loss, model_accuracy = nn_last.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 5s 5ms/step - loss: 0.6242 - accuracy: 0.6812
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5738 - accuracy: 0.7303
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5699 - accuracy: 0.7310
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5652 - accuracy: 0.7315
Epoch 5/50
801/804 [============================>.] - ETA: 0s - loss: 0.5610 - accuracy: 0.7328
Epoch 5: saving model to checkpoints/weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5609 - accuracy: 0.7328
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5581 - accuracy: 0.7329
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7324
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7334
Epoch 9/50
804/804 [==============================] - 2s 2ms/s

In [17]:
model_loss, model_accuracy = nn_new.evaluate(X_train_scaled, y_train, verbose=2)
model_loss2, model_accuracy2 = nn_last.evaluate(X_train_scaled,y_train,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

804/804 - 1s - loss: 0.5438 - accuracy: 0.7358 - 1s/epoch - 1ms/step
804/804 - 1s - loss: 0.5463 - accuracy: 0.7359 - 978ms/epoch - 1ms/step
Loss: 0.5437708497047424, Accuracy: 0.7357720136642456
Loss: 0.5463265180587769, Accuracy: 0.7359275221824646


In [ ]:
# The best accuracy was always between ~72% to ~74% as seen from the models above, so we will save the last one into h5

In [22]:
nn_new.load_weights("checkpoints/weights.50.hdf5")
# Export model
nn_new.save("AlphabetSoupCharity_Optimization.h5")
# Import model to a new object
from google.colab import drive
drive.mount('/content/gdrive')
nn_imported = tf.keras.models.load_model('gdrive/My Drive/Colab Notebooks/Neural_Network_Charity_Analysis/AlphabetSoupCharity.h5')

Mounted at /content/gdrive


In [23]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5546 - accuracy: 0.7264 - 464ms/epoch - 2ms/step
Loss: 0.5545734763145447, Accuracy: 0.7264139652252197
